In [1]:
import gym
import random
import tflearn as tf
import numpy as np
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
LR = 1e-3

env = gym.make('CartPole-v0')
env.reset()
max_goal= 500
score_requirement= 50
initial_games = 10000


/home/ankit/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
def some_random_games():
    for episode in range(50):
        env.reset()
        for t in range(max_goal):
            env.render()
            action = env.action_space.sample()     #  provided in gym
            observation, reward, done, info = env.step(action)
            if done:
                break

In [4]:
#some_random_games()

In [5]:
def initial_population():
    training_data = []
    scores = []
    accepted_scores = [] 
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(max_goal):
#             env.render()
            action = random.randrange(0,2)
            observation, reward, done, info = env.step(action)
            if len(prev_observation)>0:
                game_memory.append([prev_observation,action])

            prev_observation = observation
            score+= reward
            if done:
                break
        if score>=score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1]==1:
                    output = [0,1]
                elif data[1] ==0:
                    output = [1,0]
                training_data.append([data[0],output])
                
                
        env.reset()
        scores.append(score)
    training_data_save = np.array(training_data)
#     np.save('saved.npy', training_data_save)
    print('Average accepted score:', mean(accepted_scores))
    print('Median accepted score :', median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

In [6]:
# initial_population()

In [7]:
def neural_network_model(input_size):
    network = input_data(shape=[None,input_size,1],name  = 'input')
    
    network = fully_connected(network,128,activation = 'relu')
    network = dropout(network,0.8)
    
    network = fully_connected(network,256,activation = 'relu')
    network = dropout(network,0.8)
    
    network = fully_connected(network,512,activation = 'relu')
    network = dropout(network,0.8)
    
    network = fully_connected(network,256,activation = 'relu')
    network = dropout(network,0.8)
    
    network = fully_connected(network,128,activation = 'relu')
    network = dropout(network,0.8)
    
    network = fully_connected(network,2,activation = 'softmax')
    network = regression(network,optimizer = 'adam', learning_rate=LR, 
                         loss='categorical_crossentropy',name='targets')
    
    model =  tf.DNN(network,tensorboard_dir= 'Final_log_v1_raw_training')
    
    return model

def train_model(training_data, model  = False):
    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]
    
    if not model:
        model =neural_network_model(input_size= len(X[0]))
        
    model.fit({'input':X},{'targets':y},n_epoch=3,snapshot_step=500,show_metric =True,run_id = 'openai')
    
    return model
    
    
    
    

In [8]:
training_data = initial_population()
# training_data = np.load('saved.npy')

Average accepted score: 60.544925124792016
Median accepted score : 58.0
Counter({51.0: 95, 50.0: 94, 52.0: 81, 54.0: 80, 53.0: 74, 56.0: 57, 57.0: 56, 59.0: 55, 55.0: 55, 58.0: 48, 60.0: 46, 62.0: 45, 61.0: 42, 63.0: 40, 65.0: 28, 67.0: 27, 66.0: 23, 70.0: 22, 64.0: 21, 69.0: 19, 72.0: 18, 73.0: 17, 68.0: 17, 71.0: 15, 74.0: 13, 79.0: 12, 76.0: 9, 84.0: 9, 81.0: 8, 80.0: 8, 77.0: 8, 82.0: 7, 78.0: 6, 86.0: 5, 83.0: 5, 75.0: 5, 87.0: 3, 96.0: 3, 91.0: 3, 92.0: 2, 102.0: 2, 85.0: 2, 107.0: 2, 93.0: 2, 95.0: 2, 88.0: 2, 104.0: 1, 100.0: 1, 103.0: 1, 99.0: 1, 120.0: 1, 115.0: 1, 108.0: 1, 94.0: 1, 101.0: 1})


In [9]:
model= train_model(training_data)
# model= model.load('cartpole0.model')

Training Step: 3356  | total loss: 0.69298 | time: 20.628s
| Adam | epoch: 003 | loss: 0.69298 - acc: 0.5228 -- iter: 71552/71573
Training Step: 3357  | total loss: 0.69298 | time: 20.646s
| Adam | epoch: 003 | loss: 0.69298 - acc: 0.5221 -- iter: 71573/71573
--


In [10]:
# model.save('cartpole0.model')

In [11]:
# model.load('cartpole0.model')

In [20]:
scores=[]
choices =[]

for each_game in range(200):
    score=0
    game_memory=[]
    prev_obs = []
    env.reset()
    for _ in range(max_goal):
        env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0])
        choices.append(action)
        
        new_observation,reward,done,info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation,action])
        score += reward
        if done:
            break
    scores.append(score)
print('Average Score', sum(scores)/len(scores))
print('Choices1:{}','Choice 2: {}'.format(choices.count(1)/len(choices)),choices.count(0)/len(choices))



Average Score 27.43
Choices1:{} Choice 2: 0.9794021144732045 0.02059788552679548


In [13]:
# final_scores = np.sort(scores)


In [14]:
# final_scores = np.sort(scores)

# import matplotlib.pyplot as plt
# plt.plot(np.sort(final_scores))
# plt.title('Default Cartpole')
# plt.xlabel("episode")
# plt.ylabel('average score')

Text(0, 0.5, 'average score')

In [17]:
np.save('savev1raw.npy', final_scores)
